# Analyse accuracy of Schroedinger-schemes

In [5]:
from IPython.display import display, Markdown, Latex 

%run sps_test_suite.py

In [6]:
def analyseError(
    solver,
    config,
    generateIC,
    analyticalSolution,
    N=5,
    increaseSpatialResolution = 2, 
    increaseTemporalResolution = 1, 
    debug=False,
    waveScheme = None, 
    advection = False,
    potential = None,
):
    NX = config["resolution"]
    L = config["domainSize"]
    tEnd = config["tEnd"]
    dx = L / NX
    dt = config["dt"]
    error = np.zeros(N)

    resolutions = []
    average_timesteps = []

    for i in range(N):
        config["resolution"] = NX
        config["dt"]         = dt
        sol = solver(config, generateIC)
        sol.setExternalPotentialFunction(potential)
        if waveScheme is not None:
            wavesolver = waveScheme(config, generateIC)
            wavesolver.setExternalPotentialFunction(potential)
        else:
            wavesolver = None 

        dx = sol.dx 
        nt = 0
        timesteps = []
        while sol.getTime() < tEnd:
            delt = sol.getTimeStep()
            timesteps.append(delt)
            sol.step(delt)

            nt += 1
            if waveScheme is not None:
                wavesolver.step(delt)

        if waveScheme is None:
            if advection:
                ana, _  = analyticalSolution(*sol.grid, sol.dx, sol.t)
            else:
                ana = np.abs(analyticalSolution(*sol.grid, sol.dx, sol.t)) ** 2
        else:
            ana = np.abs(wavesolver.getPsi())**2

        num = sol.getDensity()

        error[i] = np.mean(np.abs(num - ana))
        resolutions.append(dx)
        average_timesteps.append(np.mean(timesteps))


        if debug:
            print("dx ", dx, " dt ", np.mean(timesteps))
            print("error [i]: ", i, error[i])
            animation.drawFrame(sol, waveSolver = wavesolver)
            plt.show()

        NX *= increaseSpatialResolution
        NX = int(NX)
        dt /= increaseTemporalResolution
            
    return error, resolutions, average_timesteps


In [14]:
N = 4
#test_names_1d  = ['accuracy test']
test_names  = ['periodic gaussian wave packet', 'perturbation wave']

schemes_for_accuracy_test = ["phase-ho-upwind_mc", "phase-ho-upwind_albada", "phase-ho-upwind_leer", "phase-ho-upwind"]
advection  = False
waveScheme = None # wave_schemes.SpectralScheme

increaseSpatialResolution  = 1.6
increaseTemporalResolution = 1 
suffix = ""
plotRuns = False

for test_name in test_names:
    test, testConfig, potential = test_list[test_name]

    display(Markdown('#' + " Test: " + test_name))

    truncations_errors = {}

    for scheme_name in schemes_for_accuracy_test:
        scheme, schemeConfig = scheme_list[scheme_name]

        c = getBaseConfig()
        testConfig(c)
        schemeConfig(c)
        c["stencilOrder"] = 2

        p, dx, dt = analyseError(
            scheme,
            c,
            test,
            test,
            N=N,
            increaseTemporalResolution = increaseTemporalResolution, 
            increaseSpatialResolution = increaseSpatialResolution,
            debug=plotRuns,
            waveScheme= waveScheme,
            advection = advection,
            potential = potential
        )

        truncations_errors[scheme_name] = {
            "errors": p,
            "dx": dx,
            "dt": dt
        }
        filename = "data/truncation_error_" + test_name.replace(" ", "_") + "_" + scheme_name.replace(" ", "_") + suffix + ".npz"
        print(filename)
        np.savez(filename, **(truncations_errors[scheme_name]))

# Test: periodic gaussian wave packet

Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
data/truncation_error_periodic_gaussian_wave_packet_phase-ho-upwind_mc.npz
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
data/truncation_error_periodic_gaussian_wave_packet_phase-ho-upwind_albada.npz
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
data/truncation_error_periodic_gaussian_wave_packet_phase-ho-upwind_leer.npz
C

# Test: perturbation wave

tEnd 0.3183098861837907
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
data/truncation_error_perturbation_wave_phase-ho-upwind_mc.npz
tEnd 0.3183098861837907
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
data/truncation_error_perturbation_wave_phase-ho-upwind_albada.npz
tEnd 0.3183098861837907
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
Constructing muscl-upwind scheme scheme
Setting up fourier grid
